In [4]:
import matplotlib.pyplot as plt
from fastai.vision.all import *

In [5]:
time = torch.arange(0,20)
params = torch.randn(3).requires_grad_()

In [6]:
def apply_step(params,prn=True):
    
    speed = time*3 + (time-9.5)**2 + 1
    
    a,b,c = params 
    pred = a*(time**2) + b*time + c
    loss = ((pred - speed)**2).mean()
    loss.backward()
    lr = 1e-5
    params.grad
    params.data -= lr * params.grad.data
    params.grad=None
    if prn: print(loss.item())
    return pred
for i in range(10): apply_step(params)

37646.48828125
8068.16943359375
2471.02001953125
1411.8424072265625
1211.3868408203125
1173.42822265625
1166.21923828125
1164.828369140625
1164.538818359375
1164.457763671875


In [7]:
from fastai.vision.all import *
from torch.nn import functional
import numpy
path = untar_data(URLs.MNIST_SAMPLE)



threes = (path/'train/3').ls().sorted()
sevens = (path/'train/7').ls().sorted()
valid_3s = (path/'valid/3').ls().sorted()
valid_7s = (path/'valid/7').ls().sorted()

stacked_3s = [tensor(Image.open(o)) for o in threes]
stacked_7s = [tensor(Image.open(o)) for o in sevens]
stacked_valid3s = [tensor(Image.open(o)) for o in valid_3s]
stacked_valid7s = [tensor(Image.open(o)) for o in valid_7s]
stacked_3s = torch.stack(stacked_3s).float()/255
stacked_7s = torch.stack(stacked_7s).float()/255
stacked_valid_3s = torch.stack(stacked_valid3s).float()/255
stacked_valid_7s = torch.stack(stacked_valid7s).float()/255
average_3s = (stacked_3s).mean(0)
average_7s = (stacked_7s).mean(0)
average_valid_3s = (stacked_valid_3s).mean(0)
average_valid_7s = (stacked_valid_7s).mean(0)

def L1_loss(average,real):
    result = (average - real).abs().mean()
    return result

def mean_sq_error_loss(average,real):    
    result = ((average-real)**2).sqrt().mean()
    return result

L1_loss(average_3s,stacked_3s[0])
L1_loss(average_7s,stacked_7s[0])
L1_loss(average_valid_3s,stacked_valid_3s[0])
L1_loss(average_valid_7s,stacked_valid_7s[0])

mean_sq_error_loss(average_3s,stacked_3s[0])
mean_sq_error_loss(average_7s,stacked_7s[0])
mean_sq_error_loss(average_valid_3s,stacked_valid_3s[0])
mean_sq_error_loss(average_valid_7s,stacked_valid_7s[0])

F.l1_loss(average_3s,stacked_3s[0])
F.l1_loss(average_7s,stacked_7s[0])
F.l1_loss(average_valid_3s,stacked_valid_3s[0])
F.l1_loss(average_valid_7s,stacked_valid_7s[0])

F.mse_loss(average_3s,stacked_3s[0])
F.mse_loss(average_7s,stacked_7s[0])
F.mse_loss(average_valid_3s,stacked_valid_3s[0])
F.mse_loss(average_valid_7s,stacked_valid_7s[0])


tensor(0.0296)

In [8]:
train_x = torch.cat([stacked_3s,stacked_7s]).view(-1,28*28)
train_y = tensor([0]*len(threes) + [1]*len(sevens)).unsqueeze(1)

dset = list(zip(train_x,train_y))
x,y=dset[0]
valid_x = torch.cat([stacked_valid_3s,stacked_valid_7s]).view(-1,28*28)
valid_y = tensor([0]*len(valid_3s) + [1]*len(valid_7s)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

# initialize paramters
 
def init_params(size,std=1.0): 
    return (torch.randn(size)*std).requires_grad_() 

weights = init_params(28*28,1)
bias = init_params(1)

def linear1(xb):
    return xb@weights + bias

preds = linear1(train_x)
corrects = (preds>0.0).float() == train_y
corrects.float().mean().item()
weights[0] *= 1.0001
preds = linear1(train_x)
((preds>0.0).float() == train_y.float().mean().item())

def sigmoid(x):
    return 1/(1+torch.exp(-x))

def mnist_loss(predictions,targets):
    predictions=predictions.sigmoid()
    return torch.where(targets==1,1-predictions,predictions).mean()

loss = mnist_loss(preds,train_y)


In [9]:
coll = range(15)
dl = DataLoader(coll,batch_size=5,shuffle=True)
list(dl)

ds = L(enumerate(string.ascii_lowercase))
dl = DataLoader(ds,batch_size=6,shuffle=True)
list(dl)

[(tensor([ 9, 20, 13, 17, 25,  7]), ('j', 'u', 'n', 'r', 'z', 'h')),
 (tensor([22,  8, 11, 19,  4, 14]), ('w', 'i', 'l', 't', 'e', 'o')),
 (tensor([ 6,  1, 24, 23,  0, 12]), ('g', 'b', 'y', 'x', 'a', 'm')),
 (tensor([ 2, 10, 16,  3, 18, 21]), ('c', 'k', 'q', 'd', 's', 'v')),
 (tensor([15,  5]), ('p', 'f'))]

In [10]:
weights = init_params(28*28,1)
bias = init_params(1)
dl = DataLoader(dset,batch_size = 256)
xb,yb = first(dl)
valid_dl = DataLoader(valid_dset,batch_size=256)
batch = train_x[:4]
preds = linear1(batch)
loss = mnist_loss(preds,train_y[:4])
loss.backward()

In [11]:
def calc_grad(xb,yb,model):
    preds=model(xb)
    loss = mnist_loss(preds,yb)
    loss.backward()

In [12]:
weights.grad.zero_()
bias.grad.zero_()
calc_grad(batch,train_y[:4],linear1)
weights.grad.mean(),bias.grad

(tensor(0.0090), tensor([0.0629]))

In [13]:
def train_epoch(model,lr,params):
    for xb,yb in dl:
        calc_grad(xb,yb,model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()
            
def batch_accuracy(xb,yb):
    preds = xb.sigmoid()
    correct = (preds>.5)==yb
    return correct.float().mean()


def validate_epoch(model):
    accs = [batch_accuracy(model(xb),yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(),4)


In [14]:
batch_accuracy(linear1(batch),train_y[:4])
lr = .1
params = weights,bias
train_epoch(linear1,lr,params)
validate_epoch(linear1)

for i in range(20):
    train_epoch(linear1,lr,params)
    print(validate_epoch(linear1), end = ' ')

0.8412 0.8637 0.8859 0.8966 0.9077 0.9131 0.9185 0.9218 0.9252 0.9281 0.931 0.9319 0.9339 0.9348 0.9358 0.9363 0.9368 0.9373 0.9382 0.9397 

In [15]:
linear_model = nn.Linear(28*28,1)
w,b = linear_model.parameters()

class BasicOptim:
    
    def __init__(self,params,lr): 
        self.params,self.lr = list(params),lr
        
    def step(self,*args,**kwargs):
        for p in self.params:
            p.data-=p.grad.data *self.lr
    
    def zero_grad(self,*args,**kwargs):
        for p in self.params:
            p.grad = None

In [16]:
lr=.1
opt = BasicOptim(linear_model.parameters(),lr)
def train_epoch(model):
    for xb,yb in dl:
        calc_grad(xb,yb,model)
        opt.step()
        opt.zero_grad()
        
validate_epoch(linear_model)

def train_model(model,epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model),end=' ')
        
train_model(linear_model,20)

dl = DataLoaders(dl,valid_dl)
learn = Learner(dl,nn.Linear(28*28,1),opt_func=SGD,
               loss_func=mnist_loss, metrics=batch_accuracy)

learn.fit(10,lr=lr)

def simple_net(xb):
    res = xb@w1 + b1
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res

w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

0.5439 0.8647 0.9336 0.9526 0.9634 0.9638 0.9658 0.9663 0.9678 0.9692 0.9707 0.9717 0.9717 0.9721 0.9726 0.9726 0.9726 0.9726 0.9731 0.9731 

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.196875,0.365177,0.562316,00:00
1,0.115353,0.179271,0.876840,00:00
2,0.081332,0.107115,0.934249,00:00
3,0.064630,0.080330,0.954367,00:00
4,0.055403,0.067099,0.962218,00:00
5,0.049734,0.059282,0.962709,00:00
6,0.045899,0.054109,0.965653,00:00
7,0.043090,0.050414,0.967125,00:00
8,0.040904,0.047627,0.968106,00:00
9,0.039129,0.045438,0.970069,00:00
